In [1]:
import pandas as pd
import numpy as np

In [2]:
# 1. Load rolling windows sequence (NumPy array)
sequences = np.load('rolling_window_sequences.npy')
print(f"Loaded rolling window sequences with shape: {sequences.shape}")

Loaded rolling window sequences with shape: (17731, 30, 66)


In [3]:
# 2. Load sequence metadata (engine_id, cycle)
metadata_df = pd.read_csv('sequence_metadata.csv')
print(f"Loaded sequence metadata with shape: {metadata_df.shape}")
metadata_df.head()

Loaded sequence metadata with shape: (17731, 2)


,engine_id,cycle
0,1,30.0
1,1,31.0
2,1,32.0
3,1,33.0
4,1,34.0


In [4]:
# 3. Calculate Remaining Useful Life (RUL) per sequence

# Find max cycle per engine in metadata
max_cycle_per_engine = metadata_df.groupby('engine_id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['engine_id', 'max_cycle']
max_cycle_per_engine.head()

,engine_id,max_cycle
0,1,192.0
1,2,287.0
2,3,179.0
3,4,189.0
4,5,269.0


In [5]:
# Merge max cycle info with metadata
metadata_df = metadata_df.merge(max_cycle_per_engine, on='engine_id', how='left')
metadata_df.head()

,engine_id,cycle,max_cycle
0,1,30.0,192.0
1,1,31.0,192.0
2,1,32.0,192.0
3,1,33.0,192.0
4,1,34.0,192.0


In [6]:
# Compute RUL
metadata_df['RUL'] = metadata_df['max_cycle'] - metadata_df['cycle']

In [7]:
metadata_df.head()

,engine_id,cycle,max_cycle,RUL
0,1,30.0,192.0,162.0
1,1,31.0,192.0,161.0
2,1,32.0,192.0,160.0
3,1,33.0,192.0,159.0
4,1,34.0,192.0,158.0


In [8]:
# Optional: Cap RUL values
rul_cap = 130
metadata_df['RUL'] = metadata_df['RUL'].clip(upper=rul_cap)

# Drop max_cycle column if not needed further
metadata_df.drop(columns=['max_cycle'], inplace=True)

metadata_df.head()

,engine_id,cycle,RUL
0,1,30.0,130.0
1,1,31.0,130.0
2,1,32.0,130.0
3,1,33.0,130.0
4,1,34.0,130.0


In [9]:
# 4. Validate by inspecting example engine(s)
engines_to_check = metadata_df['engine_id'].unique()[:3]  # first 3 engines
for engine in engines_to_check:
    sample = metadata_df[metadata_df['engine_id'] == engine][['cycle', 'RUL']]
    print(f"\nEngine {engine} - Cycles and RULs:")
    print(sample.head(10))
    print(sample.tail(10))


Engine 1 - Cycles and RULs:
   cycle    RUL
0   30.0  130.0
1   31.0  130.0
2   32.0  130.0
3   33.0  130.0
4   34.0  130.0
5   35.0  130.0
6   36.0  130.0
7   37.0  130.0
8   38.0  130.0
9   39.0  130.0
     cycle  RUL
153  183.0  9.0
154  184.0  8.0
155  185.0  7.0
156  186.0  6.0
157  187.0  5.0
158  188.0  4.0
159  189.0  3.0
160  190.0  2.0
161  191.0  1.0
162  192.0  0.0

Engine 2 - Cycles and RULs:
     cycle    RUL
163   30.0  130.0
164   31.0  130.0
165   32.0  130.0
166   33.0  130.0
167   34.0  130.0
168   35.0  130.0
169   36.0  130.0
170   37.0  130.0
171   38.0  130.0
172   39.0  130.0
     cycle  RUL
411  278.0  9.0
412  279.0  8.0
413  280.0  7.0
414  281.0  6.0
415  282.0  5.0
416  283.0  4.0
417  284.0  3.0
418  285.0  2.0
419  286.0  1.0
420  287.0  0.0

Engine 3 - Cycles and RULs:
     cycle    RUL
421   30.0  130.0
422   31.0  130.0
423   32.0  130.0
424   33.0  130.0
425   34.0  130.0
426   35.0  130.0
427   36.0  130.0
428   37.0  130.0
429   38.0  130.0
430   3

In [10]:
# 5. Save updated metadata with RUL target
metadata_df.to_csv('sequence_metadata_with_RUL.csv', index=False)
print("Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'")

Saved updated metadata with RUL as 'sequence_metadata_with_RUL.csv'
